# Green and Red Candle Project
## Coded by Victor Zheng & Supervised under Mohammed Hasan

## Progress
#### 04-17-2021: Debugged last index dateTimeMaxHTP for why it was (0,0,0)
#### 04-18-2021: Worked on getting dateTimeMinLTP - the minimum LTP for a paritcular row in dateTimeMaxHTP
#### 04-19-2021-A: Debugged maxHTPTime == "9:37:00 AM": case - it doesn't select the minLTP after maxtime, it selects the LTP at 9:37AM
#### 04-19-2021-B: # For each date, found the dropPerc and map it to number of occurrence and found highest occurrence to find buyPriceGeneral. Then found the profitPerc and mapped it to number of occurences and found highest occurrence to know what protit % we will accept to limit sell at


In [112]:
import pandas as pd
import numpy as np
import math
#df = pd.read_csv('Jan 2020 - Dec 2020 merged SPY - Copy.csv')
df = pd.read_csv('Jan 2020 - Dec 2020 merged SPY.csv')
df

,Date,TimeBarStart,Ticker,CallPut,Strike,ExpirationDate,OpenTradePrice,HighTradePrice,LowTradePrice,CloseTradePrice,UnderOpenBidPrice,UnderOpenAskPrice,UnderCloseBidPrice,UnderCloseAskPrice,VWAP,Volume,TotalTrades
0,20200103,9:30:00 AM,SPY,P,320.5,20200103,0.57,0.60,0.47,0.47,321.13,321.15,321.55,321.56,0.5865,241,17
1,20200103,9:30:00 AM,SPY,P,305.0,20200103,0.01,0.01,0.01,0.01,321.13,321.16,321.13,321.16,0.0100,6,1
2,20200103,9:30:00 AM,SPY,P,317.0,20200103,0.12,0.12,0.10,0.10,321.18,321.20,321.33,321.34,0.1018,11,2
3,20200103,9:30:00 AM,SPY,P,313.0,20200103,0.02,0.02,0.02,0.02,321.44,321.45,321.44,321.45,0.0200,1,1
4,20200103,9:30:00 AM,SPY,P,321.5,20200103,0.98,0.98,0.82,0.82,321.14,321.16,321.47,321.48,0.9241,279,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300055,20200311,4:14:00 PM,SPY,C,274.0,20200311,1.06,1.11,1.02,1.05,274.65,274.71,274.62,274.68,1.0830,23,7
300056,20200311,4:14:00 PM,SPY,C,271.0,20200311,3.75,3.75,3.75,3.75,274.67,274.71,274.67,274.71,3.7500,1,1
300057,20200311,4:14:00 PM,SPY,C,276.0,20200311,0.27,0.28,0.17,0.25,274.70,274.74,274.59,274.60,0.2406,17,8
300058,20200311,4:14:00 PM,SPY,C,282.0,20200311,0.01,0.01,0.01,0.01,274.54,274.59,274.54,274.59,0.0100,3,2


## Get minimum value for call/put between 9:46AM and the timecallmax for that day

In [113]:
#function that compares if time 1 is < time 2
def beforeTime(time1,time2):
    
    if len(time1) != 11:
        time1 = "0" + time1
    if len(time2) != 11:
        time2 = "0" + time2
    
    #check last 2 chars to see if AM or PM
    if time1[-2:] == "AM":
        t1IsAM = True
    else:
        t1IsAM = False
    if time2[-2:] == "AM":
        t2IsAM = True
    else:
        t2IsAM = False
    
    if t1IsAM and not t2IsAM:
        return True
    elif t1IsAM and t2IsAM:
        return True if time1 < time2 else False
    elif not t1IsAM and t2IsAM:
        return False
    else: #t1 is pm and t2 is pm
        return True if time1 < time2 else False

#sample test cases - expected TFFF
print(beforeTime("10:05:00 AM","3:59:00 PM"), beforeTime("10:05:00 PM","3:59:00 PM"), beforeTime("10:05:00 AM","3:59:00 AM"),beforeTime("10:05:00 PM","3:59:00 AM"))

def afterTime(time1,time2):
    if time1 == time2:
        return False
    return not beforeTime(time1,time2)

True False False False


### Starting price difference - green (increase) and red (decrease) indicators for the first 5 mins of each date

In [114]:
dateTimeCandleSP = [] #(date,time,color,StrikePrice,StockPrice)
date = df.iloc[0]["Date"]
openPrice = df.iloc[0]["UnderOpenAskPrice"]
skipDate = False
#indices: 0=Date, 1=TimeBarStart, 13= UnderCloseAskPrice, 11=UnderOpenAskPrice
for index,row in df.iterrows():
    #we are looking for the UnderOpenAskPrice(UOAP) at 9:30:00 AM (the first time it occurs) 
    #we are looking for the UnderOpenAskPrice (UOAP) at 9:36:00 AM (the last time it occurs)
    #then we subtract the 9:35:00 AM time by the 9:30:00 AM time
    
    if date == row[0]:
        if skipDate:
            continue
        #lastPrice = row["UnderCloseAskPrice"]
        prevRow = row
        #if time is past 9:35AM, then use the latest 9:35AM (if it doesn't exist, then use the current row's time)
        time = row[1]
        if not beforeTime(time,"9:36:00 AM"):
            #calculate starting price difference
            
            #if 9:35 exists
            if prevRow[1] == "9:35:00 AM":
                #we take the UnderCloseAskPrice
                lastPrice = prevRow[13]
            else:
                #if 9:35 doesn't exist
                #if 9:36 exists, then take the open price, otherwise skip this date
                if time == "9:36:00 AM":
                    lastPrice = row[11]
                else:
                    skipDate = True
                    continue
            diff = lastPrice - openPrice
            if math.isnan(diff):
                #either openPrice or lastPrice doesn't exist
                skipDate = True
                continue
            #print("date={},lastPrice={}".format(date,lastPrice))
            if diff >= 0:
                #green start
                dateTimeCandleSP.append((date,time,"GREEN",math.ceil(lastPrice),lastPrice))
            else:
                dateTimeCandleSP.append((date,time,"RED",math.floor(lastPrice),lastPrice))
            skipDate = True
    else:
        #we found new date, get the green/red bar for the previous date, reinitialize some variables
        date = row[0]
        openPrice = row[11]
        skipDate = False

In [115]:
#(date,time,color,StrikePrice,StockPrice)
dateTimeCandleSP

[(20200103, '9:36:00 AM', 'GREEN', 323, 322.41),
 (20200106, '9:36:00 AM', 'GREEN', 322, 321.24),
 (20200108, '9:36:00 AM', 'GREEN', 324, 323.4),
 (20200110, '9:36:00 AM', 'RED', 326, 326.84),
 (20200113, '9:36:00 AM', 'RED', 326, 326.18),
 (20200115, '9:36:00 AM', 'GREEN', 329, 328.08),
 (20200117, '9:36:00 AM', 'RED', 331, 331.45),
 (20200121, '9:36:00 AM', 'GREEN', 332, 331.4),
 (20200122, '9:36:00 AM', 'GREEN', 333, 332.44),
 (20200124, '9:36:00 AM', 'RED', 332, 332.17),
 (20200127, '9:36:00 AM', 'GREEN', 324, 323.77),
 (20200129, '9:36:00 AM', 'RED', 328, 328.41),
 (20200131, '9:36:00 AM', 'RED', 326, 326.16),
 (20200203, '9:36:00 AM', 'GREEN', 325, 324.16),
 (20200205, '9:36:00 AM', 'RED', 331, 331.93),
 (20200207, '9:36:00 AM', 'RED', 332, 332.26),
 (20200210, '9:36:00 AM', 'GREEN', 333, 332.14),
 (20200212, '9:36:00 AM', 'GREEN', 338, 337.03),
 (20200214, '9:36:00 AM', 'GREEN', 338, 337.51),
 (20200218, '9:36:00 AM', 'GREEN', 337, 337.0),
 (20200219, '9:36:00 AM', 'GREEN', 338,

In [116]:
dtcspIdx = 0
dfStartIdx = 0
dateWithOTP = [] #(date, time, CallPut, SP, OTP)
for dtcsp in dateTimeCandleSP:
    #print("dateWithOTP: {}".format(dateWithOTP))
    date, time, color, strikeP = dtcsp[0],dtcsp[1],dtcsp[2], dtcsp[3]
    for index,row in df.iloc[dfStartIdx:].iterrows():
        if color == "GREEN":
            #This is call, get OpenTradePrice(OTP) at 9:36AM
            #Match Date, Time, Call=C, StrikePrice
            if row[0] == date and row[1] == time and row[3] == "C" and row[4] == strikeP:
                dateWithOTP.append((date,time,"C",strikeP,row[6]))
                #update for next iteration of dtcsp
                dfStartIdx = index + 1
                break
        else:
            #This is put, get OpenTradePrice(OTP) at 9:36AM
            #Match Date, Time, Call=P, StrikePrice
            if row[0] == date and row[1] == time and row[3] == "P" and row[4] == strikeP:
                dateWithOTP.append((date,time,"P",strikeP,row[6]))
                #update for next iteration of dtcsp
                dfStartIdx = index + 1
                break

#if the candle is green then we want to buy a call
#if call then you look for the UnderOpenAskPrice at 9:36:00 AM and round that up to next whole number, this gets the Call_SP
#then we look for the Call_SP and see what the OpenTradePrice is at 9:36:00 AM


#if the candle is red then we want to buy a put
#if put then you look for the UnderOpenAskPrice at 9:36:00 AM and round that down to next whole number, this gets the Put_SP
#then we look for the Put_SP and see what the OpenTradePrice is at 9:36:00 AM

In [117]:
#(date, time, CallPut, SP, OTP)
print("(date, time,CallPut, SP, OTP)")
dateWithOTP

(date, time,CallPut, SP, OTP)


[(20200103, '9:36:00 AM', 'C', 323, 0.43),
 (20200106, '9:36:00 AM', 'C', 322, 0.31),
 (20200108, '9:36:00 AM', 'C', 324, 0.39),
 (20200110, '9:36:00 AM', 'P', 326, 0.21),
 (20200113, '9:36:00 AM', 'P', 326, 0.44),
 (20200115, '9:36:00 AM', 'C', 329, 0.19),
 (20200117, '9:36:00 AM', 'P', 331, 0.3),
 (20200121, '9:36:00 AM', 'C', 332, 0.23),
 (20200122, '9:36:00 AM', 'C', 333, 0.23),
 (20200124, '9:36:00 AM', 'P', 332, 0.36),
 (20200127, '9:36:00 AM', 'C', 324, 1.06),
 (20200129, '9:36:00 AM', 'P', 328, 0.69),
 (20200131, '9:36:00 AM', 'P', 326, 0.87),
 (20200203, '9:36:00 AM', 'C', 325, 0.5),
 (20200205, '9:36:00 AM', 'P', 331, 0.43),
 (20200207, '9:36:00 AM', 'P', 332, 0.71),
 (20200210, '9:36:00 AM', 'C', 333, 0.31),
 (20200212, '9:36:00 AM', 'C', 338, 0.14),
 (20200214, '9:36:00 AM', 'C', 338, 0.37),
 (20200218, '9:36:00 AM', 'C', 337, 0.65),
 (20200219, '9:36:00 AM', 'C', 338, 0.45),
 (20200221, '9:36:00 AM', 'P', 335, 0.98),
 (20200224, '9:36:00 AM', 'C', 325, 1.28),
 (20200226, '

#### For each date,SP,CallPut in dateWithOTP, find the maximum of the hightradeprice and time (b/w 9:37:00 AM to 3:59:00 PM

In [118]:
dfStartIdx = 0
#Date	TimeBarStart	Ticker	CallPut	Strike	ExpirationDate	OpenTradePrice	HighTradePrice	LowTradePrice	CloseTradePrice	
dateTimeMaxHTP = [(0,0,0,0,0) for _ in range(len(dateWithOTP))] #[date,time,CallPut,SP,maxHTP]
prevDate = dateWithOTP[0][0]
N = len(df)
curDfLen = N
for dOTPidx, (date,_,CallPut,sp,_) in enumerate(dateWithOTP):
    #print("Checking for date={},CallPut={},sp={}".format(date,CallPut,sp))
    maxHTP = -1
    doSameDate = False
    for index,row in df.iloc[dfStartIdx:].iterrows():
        #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
        #print("htp={},maxHTP={},time={}".format(row[7],maxHTP,row[1]))
        if index == N-1: #index == N - dfStartIdx - 2:
            #needed this one here to check for end index, which can contain time inside the timeframe we want
            #print("if4.1-----------------------------------------")
            #case 1 - we're done with prev date
            dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

            #initialize variables for next iteration of dateWithOTP
            dfStartIdx = index + 1
            prevDate = row[0]
            maxHTP = -1
            break
            
        if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],"3:59:00 PM"):
            if index == N-1: #index == N - dfStartIdx - 2:
                #need this one here again because the last index may have time that is outside of timeframe
                #print("if4.1-----------------------------------------")
                #case 1 - we're done with prev date
                dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)
            
                #initialize variables for next iteration of dateWithOTP
                dfStartIdx = index + 1
                prevDate = row[0]
                maxHTP = -1
                break
            continue
        elif date == row[0] and CallPut == row[3] and sp == row[4]:
            #print("if1")
            #print("row: {}".format(list(row)))
            #Check if there's a new max HTP
            #print("htp={},maxHTP={},time={}".format(row[7],maxHTP,row[1]))
            if row[7] > maxHTP:
                #print("if2")
                #print("update for maxHTP: {}".format(row[7]))
                maxHTP = row[7]
                maxHTPTime = row[1]
        elif date != row[0]:
            #print("if3")
            #date doesn't match, so we're done searching for previous date OR this date has no value in df
            if maxHTP != -1: #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMaxHTP  
                #print("if4")
                #case 1 - we're done with prev date
                dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)
            
                #initialize variables for next iteration of dateWithOTP
                dfStartIdx = index + 1
                prevDate = row[0]
                maxHTP = -1
                break
            elif index == N-1: #index == N - dfStartIdx - 2:
                #print("if4.1-----------------------------------------")
                #case 1 - we're done with prev date
                dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)
            
                #initialize variables for next iteration of dateWithOTP
                dfStartIdx = index + 1
                prevDate = row[0]
                maxHTP = -1
                break
            else:
                #print('if5')
                #Case 2 - date has no value in df, so doesn't exist in dateWithOTP
                #keep iterating until we past this date and until we found the date that matches dateWithOTP
                dfStartIdx = index + 1
                #print("dfStartIdx: {}".format(dfStartIdx))
                for indexb,rowb in df.iloc[dfStartIdx:].iterrows():
                    if rowb[0] == date:
                        #print("if6")
                        prevDate = rowb[0]
                        dfStartIdx = indexb
                        doSameDate = True #found matching date so now iterate from that as startIdx to cover it instead of using current for loop
                        break
                if doSameDate:
                    #print("if7")
                    break
            

    if doSameDate:
        #print("if7.2")
        for index,row in df.iloc[dfStartIdx:].iterrows():
            #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
            if index == N-1: #N - dfStartIdx - 2: # 80352 - 41175 # - 1
                    #print("if11")
                    #reached the last index in df, so update for maxHTP
                    #case 1 - we're done with prev date
                    dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                    #initialize variables for next iteration of dateWithOTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    maxHTP = -1
                    break
                    
            if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],"3:59:00 PM"):
                if index == N-1: #N - dfStartIdx - 2: # 80352 - 41175 # - 1
                    #print("if11")
                    #reached the last index in df, so update for maxHTP
                    #case 1 - we're done with prev date
                    dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                    #initialize variables for next iteration of dateWithOTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    maxHTP = -1
                    break
                continue
            elif date == row[0] and CallPut == row[3] and sp == row[4]:
                #print("if8")
                #print("row: {}".format(list(row)))
                #Check if there's a new max HTP
                #print("htp={},maxHTP={},time={}".format(row[7],maxHTP,row[1]))
                if row[7] > maxHTP:
                    #print("if9")
                    #print("update for maxHTP: {}".format(row[7]))
                    maxHTP = row[7]
                    maxHTPTime = row[1]
            elif date != row[0]:
                #date doesn't match, so we're done searching for previous date OR this date has no value in df
                #print("if10")
                if maxHTP != -1: #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMaxHTP 
                    #case 1 - we're done with prev date
                    dateTimeMaxHTP[dOTPidx] = (prevDate,maxHTPTime,CallPut,sp,maxHTP)

                    #initialize variables for next iteration of dateWithOTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    maxHTP = -1
                    break
            

In [119]:
dateTimeMaxHTP

[(20200103, '1:03:00 PM', 'C', 323, 0.7),
 (20200106, '3:59:00 PM', 'C', 322, 1.65),
 (20200108, '3:29:00 PM', 'C', 324, 1.78),
 (20200110, '3:42:00 PM', 'P', 326, 0.81),
 (20200113, '9:42:00 AM', 'P', 326, 0.55),
 (20200115, '11:04:00 AM', 'C', 329, 0.44),
 (20200117, '9:40:00 AM', 'P', 331, 0.41),
 (20200121, '11:57:00 AM', 'C', 332, 0.36),
 (20200122, '9:48:00 AM', 'C', 333, 0.32),
 (20200124, '2:24:00 PM', 'P', 332, 4.6),
 (20200127, '9:52:00 AM', 'C', 324, 1.49),
 (20200129, '10:08:00 AM', 'P', 328, 1.9),
 (20200131, '3:42:00 PM', 'P', 326, 5.15),
 (20200203, '10:30:00 AM', 'C', 325, 1.5),
 (20200205, '10:41:00 AM', 'P', 331, 0.85),
 (20200207, '10:04:00 AM', 'P', 332, 0.81),
 (20200210, '3:59:00 PM', 'C', 333, 1.64),
 (20200212, '9:40:00 AM', 'C', 338, 0.2),
 (20200214, '9:40:00 AM', 'C', 338, 0.32),
 (20200218, '9:42:00 AM', 'C', 337, 0.65),
 (20200219, '2:45:00 PM', 'C', 338, 1.1),
 (20200221, '1:55:00 PM', 'P', 335, 2.41),
 (20200224, '10:01:00 AM', 'C', 325, 1.79),
 (20200226

#### Find the minimum of the option (LowTradePrice) from 9:37:00 AM to the maxHTP time
#### If the max time occurs at 9:37:00 AM then find the minimum LowTradePrice of the option from 9:37:00 AM to the end of the day

In [135]:
dfStartIdx = 0
#Date	TimeBarStart	Ticker	CallPut	Strike	ExpirationDate	OpenTradePrice	HighTradePrice	LowTradePrice	CloseTradePrice	
dateTimeMinLTP = [(0,0,0,0,0) for _ in range(len(dateTimeMaxHTP))] #[date,time,CallPut,SP,minLTP]
prevDate = dateTimeMaxHTP[0][0]
N = len(df)
curDfLen = N
for dTMidx, (date,maxHTPTime,CallPut,sp,_) in enumerate(dateTimeMaxHTP):
    
    while int(prevDate) < date:
        #if the row's date is not part of our dateTimeMaxHTP, then we keep moving to next row
        dfStartIdx += 1
        if dfStartIdx == N - 1:
            break
        prevDate = df.iloc[dfStartIdx][0]


    #print("Checking for date={},maxHTPTime={},CallPut={},sp={}".format(date,maxHTPTime,CallPut,sp))
    minLTP = float("inf")
    doSameDate = False
    
    if maxHTPTime == "9:37:00 AM":
        #print("inside maxHTPTime == 9:37:00 AM")
        #if the max time occurs at 9:37:00 AM then find the minimum LowTradePrice of the option from 9:37:00 AM to the end of the day
        for index,row in df.iloc[dfStartIdx:].iterrows():
            #print("index,dfStartIdx,N-dfStartIdx-1,time: {},{},{},{}".format(index,dfStartIdx,N-dfStartIdx-1,row[1]))
            #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
            if index == N-1: #index == N - dfStartIdx - 2:
                #case 1 - we're done with prev date
                #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                #initialize variables for next iteration of dateTimeMaxHTP
                dfStartIdx = index + 1
                prevDate = row[0]
                minLTP = float("inf")
                break

            #want 9:37AM to end of day
            #otherwise toss out this row
            if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],"3:59:00 PM"):
                if index == N-1: #index == N - dfStartIdx - 2:
                    #case 1 - we're done with prev date
                    #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                    dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                    #initialize variables for next iteration of dateTimeMaxHTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    minLTP = float("inf")
                    break
                continue
            elif date == row[0] and CallPut == row[3] and sp == row[4]:
                #print("row: {}".format(list(row)))
                #Check if there's a new max HTP
                #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                if row[8] < minLTP:
                    #print("update for minLTP: {}".format(row[8]))
                    minLTP = row[8]
                    minLTPTime = row[1]
            elif date != row[0]:
                #date doesn't match, so we're done searching for previous date OR this date has no value in df
                if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP  
                    #case 1 - we're done with prev date
                    #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                    dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                    #initialize variables for next iteration of dateTimeMaxHTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    minLTP = float("inf")
                    break
                elif index == N-1: #index == N - dfStartIdx - 2:
                    #case 1 - we're done with prev date
                    #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                    dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                    #initialize variables for next iteration of dateTimeMaxHTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    minLTP = float("inf")
                    break
                else:
                    #print('if5 - date has no value in df')
                    #Case 2 - date has no value in df, so doesn't exist in dateTimeMaxHTP
                    #keep iterating until we past this date and until we found the date that matches dateTimeMaxHTP
                    dfStartIdx = index + 1
                    #print("dfStartIdx: {}".format(dfStartIdx))
                    for indexb,rowb in df.iloc[dfStartIdx:].iterrows():
                        if rowb[0] == date:
                            prevDate = rowb[0]
                            dfStartIdx = indexb
                            doSameDate = True #found matching date so now iterate from that as startIdx to cover it instead of using current for loop
                            break
                    if doSameDate:
                        break
            

            if doSameDate:
                for index,row in df.iloc[dfStartIdx:].iterrows():
                    #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
                    if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP  
                        #case 1 - we're done with prev date
                        #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                        dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                        #initialize variables for next iteration of dateTimeMaxHTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        minLTP = float("inf")
                        break

                    #want 9:37AM to end of day
                    #otherwise toss out this row
                    if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],"3:59:00 PM"):
                        if index == N-1: #N - dfStartIdx - 2: # 80352 - 41175 # - 1
                            #reached the last index in df, so update for minLTP
                            #case 1 - we're done with prev date
                            #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                            dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                            #initialize variables for next iteration of dateTimeMaxHTP
                            dfStartIdx = index + 1
                            prevDate = row[0]
                            minLTP = float("inf")
                            break
                        continue
                    elif date == row[0] and CallPut == row[3] and sp == row[4]:
                        #print("row: {}".format(list(row)))
                        #Check if there's a new max HTP
                        #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                        if row[8] < minLTP:
                            #print("update for minLTP: {}".format(row[8]))
                            minLTP = row[8]
                            minLTPTime = row[1]
                    elif date != row[0]:
                        #date doesn't match, so we're done searching for previous date OR this date has no value in df
                        if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP 
                            #case 1 - we're done with prev date
                            #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                            dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                            #initialize variables for next iteration of dateTimeMaxHTP
                            dfStartIdx = index + 1
                            prevDate = row[0]
                            minLTP = float("inf")
                            break
    else:
        #print("NOT inside maxHTPTime == 9:37:00 AM")
        #max occurs after 9:37AM - find the minimum of the option (LowTradePrice) from 9:37:00 AM to the max HTP time
        for index,row in df.iloc[dfStartIdx:].iterrows():
            #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
            #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
            if index == N-1: #index == N - dfStartIdx - 2:
                #case 1 - we're done with prev date
                #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                #initialize variables for next iteration of dateTimeMaxHTP
                dfStartIdx = index + 1
                prevDate = row[0]
                minLTP = float("inf")
                break

            #want 9:37AM to maxHTP time
            #toss out others
            if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],maxHTPTime):
                if index == N-1: #index == N - dfStartIdx - 2:
                    #case 1 - we're done with prev date
                    #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                    dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                    #initialize variables for next iteration of dateTimeMaxHTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    minLTP = float("inf")
                    break
                continue
            elif date == row[0] and CallPut == row[3] and sp == row[4]:
                #print("row: {}".format(list(row)))
                #Check if there's a new max HTP
                #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                if row[8] < minLTP:
                    #print("update for minLTP: {}".format(row[8]))
                    minLTP = row[8]
                    minLTPTime = row[1]
            elif date != row[0]:
                #date doesn't match, so we're done searching for previous date OR this date has no value in df
                if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP  
                    #case 1 - we're done with prev date
                    #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                    dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                    #initialize variables for next iteration of dateTimeMaxHTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    minLTP = float("inf")
                    break
                elif index == N-1: #index == N - dfStartIdx - 2:
                    #case 1 - we're done with prev date
                    #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                    dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                    #initialize variables for next iteration of dateTimeMaxHTP
                    dfStartIdx = index + 1
                    prevDate = row[0]
                    minLTP = float("inf")
                    break
                else:
                    #print('if5')
                    #Case 2 - date has no value in df, so doesn't exist in dateTimeMaxHTP
                    #keep iterating until we past this date and until we found the date that matches dateTimeMaxHTP
                    dfStartIdx = index + 1
                    #print("dfStartIdx: {}".format(dfStartIdx))
                    for indexb,rowb in df.iloc[dfStartIdx:].iterrows():
                        if rowb[0] == date:
                            prevDate = rowb[0]
                            dfStartIdx = indexb
                            doSameDate = True #found matching date so now iterate from that as startIdx to cover it instead of using current for loop
                            break
                    if doSameDate:
                        break
            

            if doSameDate:
                for index,row in df.iloc[dfStartIdx:].iterrows():
                    #print("index,dfStartIdx,N-dfStartIdx-1: {},{},{}".format(index,dfStartIdx,N-dfStartIdx-1))
                    if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP  
                        #case 1 - we're done with prev date
                        #print("prevDate={},CallPut={},sp={},minLTP={}".format(prevDate,CallPut,sp,minLTP))
                        #print("prevDate={},minLTPTime={},CallPut={},sp={},minLTP={}".format(prevDate,minLTPTime,CallPut,sp,minLTP))
                        dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                        #initialize variables for next iteration of dateTimeMaxHTP
                        dfStartIdx = index + 1
                        prevDate = row[0]
                        minLTP = float("inf")
                        break

                    if beforeTime(row[1],"9:37:00 AM") or afterTime(row[1],maxHTPTime):
                        if index == N-1: #N - dfStartIdx - 2: # 80352 - 41175 # - 1
                            #reached the last index in df, so update for minLTP
                            #case 1 - we're done with prev date
                            dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                            #initialize variables for next iteration of dateTimeMaxHTP
                            dfStartIdx = index + 1
                            prevDate = row[0]
                            minLTP = float("inf")
                            break
                        continue
                    elif date == row[0] and CallPut == row[3] and sp == row[4]:
                        #print("row: {}".format(list(row)))
                        #Check if there's a new max HTP
                        #print("htp={},minLTP={},time={}".format(row[8],minLTP,row[1]))
                        if row[8] < minLTP:
                            #print("update for minLTP: {}".format(row[8]))
                            minLTP = row[8]
                            minLTPTime = row[1]
                    elif date != row[0]:
                        #date doesn't match, so we're done searching for previous date OR this date has no value in df
                        if minLTP != float("inf"): #or index == N - dfStartIdx - 1: #the right of "or" represents last index, so we compute dateTimeMinLTP 
                            #case 1 - we're done with prev date
                            dateTimeMinLTP[dTMidx] = (prevDate,minLTPTime,CallPut,sp,minLTP)

                            #initialize variables for next iteration of dateTimeMaxHTP
                            dfStartIdx = index + 1
                            prevDate = row[0]
                            minLTP = float("inf")
                            break

In [121]:
dateWithOTP

[(20200103, '9:36:00 AM', 'C', 323, 0.43),
 (20200106, '9:36:00 AM', 'C', 322, 0.31),
 (20200108, '9:36:00 AM', 'C', 324, 0.39),
 (20200110, '9:36:00 AM', 'P', 326, 0.21),
 (20200113, '9:36:00 AM', 'P', 326, 0.44),
 (20200115, '9:36:00 AM', 'C', 329, 0.19),
 (20200117, '9:36:00 AM', 'P', 331, 0.3),
 (20200121, '9:36:00 AM', 'C', 332, 0.23),
 (20200122, '9:36:00 AM', 'C', 333, 0.23),
 (20200124, '9:36:00 AM', 'P', 332, 0.36),
 (20200127, '9:36:00 AM', 'C', 324, 1.06),
 (20200129, '9:36:00 AM', 'P', 328, 0.69),
 (20200131, '9:36:00 AM', 'P', 326, 0.87),
 (20200203, '9:36:00 AM', 'C', 325, 0.5),
 (20200205, '9:36:00 AM', 'P', 331, 0.43),
 (20200207, '9:36:00 AM', 'P', 332, 0.71),
 (20200210, '9:36:00 AM', 'C', 333, 0.31),
 (20200212, '9:36:00 AM', 'C', 338, 0.14),
 (20200214, '9:36:00 AM', 'C', 338, 0.37),
 (20200218, '9:36:00 AM', 'C', 337, 0.65),
 (20200219, '9:36:00 AM', 'C', 338, 0.45),
 (20200221, '9:36:00 AM', 'P', 335, 0.98),
 (20200224, '9:36:00 AM', 'C', 325, 1.28),
 (20200226, '

In [122]:
#[date,time,CallPut,sp,minLTP]
dateTimeMinLTP

[(20200103, '11:10:00 AM', 'C', 323, 0.2),
 (20200106, '9:38:00 AM', 'C', 322, 0.24),
 (20200108, '10:49:00 AM', 'C', 324, 0.21),
 (20200110, '11:11:00 AM', 'P', 326, 0.07),
 (20200113, '9:42:00 AM', 'P', 326, 0.34),
 (20200115, '9:45:00 AM', 'C', 329, 0.16),
 (20200117, '9:37:00 AM', 'P', 331, 0.33),
 (20200121, '11:00:00 AM', 'C', 332, 0.07),
 (20200122, '9:37:00 AM', 'C', 333, 0.22),
 (20200124, '9:59:00 AM', 'P', 332, 0.34),
 (20200127, '9:37:00 AM', 'C', 324, 1.0),
 (20200129, '9:37:00 AM', 'P', 328, 0.69),
 (20200131, '9:39:00 AM', 'P', 326, 0.6),
 (20200203, '9:41:00 AM', 'C', 325, 0.51),
 (20200205, '10:05:00 AM', 'P', 331, 0.26),
 (20200207, '9:55:00 AM', 'P', 332, 0.42),
 (20200210, '9:50:00 AM', 'C', 333, 0.26),
 (20200212, '9:37:00 AM', 'C', 338, 0.14),
 (20200214, '9:37:00 AM', 'C', 338, 0.25),
 (20200218, '9:37:00 AM', 'C', 337, 0.43),
 (20200219, '10:13:00 AM', 'C', 338, 0.42),
 (20200221, '11:55:00 AM', 'P', 335, 0.71),
 (20200224, '9:42:00 AM', 'C', 325, 1.32),
 (20200

In [123]:
dateTimeMaxHTP

[(20200103, '1:03:00 PM', 'C', 323, 0.7),
 (20200106, '3:59:00 PM', 'C', 322, 1.65),
 (20200108, '3:29:00 PM', 'C', 324, 1.78),
 (20200110, '3:42:00 PM', 'P', 326, 0.81),
 (20200113, '9:42:00 AM', 'P', 326, 0.55),
 (20200115, '11:04:00 AM', 'C', 329, 0.44),
 (20200117, '9:40:00 AM', 'P', 331, 0.41),
 (20200121, '11:57:00 AM', 'C', 332, 0.36),
 (20200122, '9:48:00 AM', 'C', 333, 0.32),
 (20200124, '2:24:00 PM', 'P', 332, 4.6),
 (20200127, '9:52:00 AM', 'C', 324, 1.49),
 (20200129, '10:08:00 AM', 'P', 328, 1.9),
 (20200131, '3:42:00 PM', 'P', 326, 5.15),
 (20200203, '10:30:00 AM', 'C', 325, 1.5),
 (20200205, '10:41:00 AM', 'P', 331, 0.85),
 (20200207, '10:04:00 AM', 'P', 332, 0.81),
 (20200210, '3:59:00 PM', 'C', 333, 1.64),
 (20200212, '9:40:00 AM', 'C', 338, 0.2),
 (20200214, '9:40:00 AM', 'C', 338, 0.32),
 (20200218, '9:42:00 AM', 'C', 337, 0.65),
 (20200219, '2:45:00 PM', 'C', 338, 1.1),
 (20200221, '1:55:00 PM', 'P', 335, 2.41),
 (20200224, '10:01:00 AM', 'C', 325, 1.79),
 (20200226

In [124]:
numValidDates = len(dateWithOTP)
mergedOtpLtpHtp = [(0,"Z",0,("Z",0),("Z",0),("Z",0))] * numValidDates #(date,CallPut,SP,(time1,OTP),(time2,minLTP),(time3,maxHTP))
for index in range(numValidDates):
    date = dateWithOTP[index][0]
    CallPut = dateWithOTP[index][2]
    SP = dateWithOTP[index][3]
    time1, OTP = dateWithOTP[index][1], dateWithOTP[index][4]
    time2, minLTP = dateTimeMinLTP[index][1], dateTimeMinLTP[index][4]
    time3, maxHTP = dateTimeMaxHTP[index][1], dateTimeMaxHTP[index][4]
    mergedOtpLtpHtp[index] = (date,CallPut,SP,(time1,OTP),(time2,minLTP),(time3,maxHTP))
    
    

In [125]:
mergedOtpLtpHtp

[(20200103,
  'C',
  323,
  ('9:36:00 AM', 0.43),
  ('11:10:00 AM', 0.2),
  ('1:03:00 PM', 0.7)),
 (20200106,
  'C',
  322,
  ('9:36:00 AM', 0.31),
  ('9:38:00 AM', 0.24),
  ('3:59:00 PM', 1.65)),
 (20200108,
  'C',
  324,
  ('9:36:00 AM', 0.39),
  ('10:49:00 AM', 0.21),
  ('3:29:00 PM', 1.78)),
 (20200110,
  'P',
  326,
  ('9:36:00 AM', 0.21),
  ('11:11:00 AM', 0.07),
  ('3:42:00 PM', 0.81)),
 (20200113,
  'P',
  326,
  ('9:36:00 AM', 0.44),
  ('9:42:00 AM', 0.34),
  ('9:42:00 AM', 0.55)),
 (20200115,
  'C',
  329,
  ('9:36:00 AM', 0.19),
  ('9:45:00 AM', 0.16),
  ('11:04:00 AM', 0.44)),
 (20200117,
  'P',
  331,
  ('9:36:00 AM', 0.3),
  ('9:37:00 AM', 0.33),
  ('9:40:00 AM', 0.41)),
 (20200121,
  'C',
  332,
  ('9:36:00 AM', 0.23),
  ('11:00:00 AM', 0.07),
  ('11:57:00 AM', 0.36)),
 (20200122,
  'C',
  333,
  ('9:36:00 AM', 0.23),
  ('9:37:00 AM', 0.22),
  ('9:48:00 AM', 0.32)),
 (20200124,
  'P',
  332,
  ('9:36:00 AM', 0.36),
  ('9:59:00 AM', 0.34),
  ('2:24:00 PM', 4.6)),
 (202001

In [126]:
value = 1 - ((2.26 - 2.22) / 2.26 * 100)
formatted_string = "{:.3f}".format(value)
print(formatted_string + "%")

-0.770%


In [134]:
import collections
dropFromOTP = collections.defaultdict(int) #part 1
buyToMax = collections.defaultdict(int) #part 2
numValidDates = len(mergedOtpLtpHtp)
for index in range(numValidDates):
    time1, otp = mergedOtpLtpHtp[index][3][0], mergedOtpLtpHtp[index][3][1]
    time2, minLTP = mergedOtpLtpHtp[index][4][0], mergedOtpLtpHtp[index][4][1]
    time3, maxHTP = mergedOtpLtpHtp[index][5][0], mergedOtpLtpHtp[index][5][1]
    if afterTime(time2,time3):
        #failure date, we lose 50%
        dropPerc = -50
    else:
        dropPerc = math.ceil((otp - minLTP) / otp * 100)
    #print(otp,minLTP,dropPerc)
    dropFromOTP[dropPerc] += 1
    #print("otp={}, minLTP={}, dropPerc={}".format(otp,minLTP,dropPerc))
    
mostOccurredDropPerc = max(dropFromOTP, key=dropFromOTP.get)
print("mostOccurredDropPerc:{}".format(mostOccurredDropPerc))

for index in range(numValidDates):
    time1, otp = mergedOtpLtpHtp[index][3][0], mergedOtpLtpHtp[index][3][1]
    time2, minLTP = mergedOtpLtpHtp[index][4][0], mergedOtpLtpHtp[index][4][1]
    time3, maxHTP = mergedOtpLtpHtp[index][5][0], mergedOtpLtpHtp[index][5][1]
    #print("otp={}, minLTP={}, mostOccurredDropPerc={}, buyPriceGeneral={}".format(otp,minLTP,mostOccurredDropPerc,buyPriceGeneral))
    
    #percent profit from buyPriceGeneral --> maxHTP
    if afterTime(time2,time3):
        buyPriceGeneral = 0
        profitPerc = -50
    else:
        buyPriceGeneral = otp - mostOccurredDropPerc/100*otp
        profitPerc = math.floor((maxHTP - buyPriceGeneral) / maxHTP * 100)
    if buyPriceGeneral*0.5 >= minLTP:
        #if our minLTP can still drop very low (past 50% our buy) then we have a failure
        profitPerc = -50
    #print("otp={}, minLTP={}, mostOccurredDropPerc={}, buyPriceGeneral={}, profitPerc={}".format(otp,minLTP,mostOccurredDropPerc,buyPriceGeneral,profitPerc))
    buyToMax[profitPerc] += 1
temp = buyToMax[-50]
del buyToMax[-50]
mostOccuredBuyProfit = max(buyToMax, key=buyToMax.get) #excludes -50
print("mostOccuredBuyProfit:{}".format(mostOccuredBuyProfit))
buyToMax[-50] = temp

mostOccurredDropPerc:6
mostOccuredBuyProfit:32


In [133]:
print("dropFromOTP: {}".format(dropFromOTP))
print("buyToMax: {}".format(buyToMax))
print("mostOccurredDropPerc: {}".format(mostOccurredDropPerc))
print("mostOccuredBuyProfit: {}".format(mostOccuredBuyProfit))

dropFromOTP: defaultdict(<class 'int'>, {54: 1, 23: 2, 47: 1, 67: 1, 16: 1, -10: 1, 70: 1, 5: 1, 6: 3, 0: 2, 32: 1, -2: 1, 40: 1, 41: 1, 17: 1, 33: 1, 34: 2, 7: 1, 28: 1, -3: 1, 65: 1, 30: 1, 2: 1, -50: 1})
buyToMax: defaultdict(<class 'int'>, {82: 2, 79: 1, 24: 1, 59: 1, 31: 1, 32: 3, 92: 1, 33: 1, 65: 1, 84: 1, 68: 1, 52: 1, 17: 1, 34: 1, -9: 1, 6: 1, 61: 2, 53: 1, 39: 1, 47: 1, -50: 5})
mostOccurredDropPerc: 6
mostOccuredBuyProfit: 32


In [ ]:
#Using the mostOccuredBuyProfit we need to look through the data again and see if the maxHTP reached that percentage, 
#if so, we profit that %, otherwise we would code it in is as -50% (which is our limit sell)
#we then see the amount of situations we got correct and which ones we got wrong and see if the strategy is profitable
#overall when used on a new set of data (validation set)